# Webscraping wiso-net
- This playbook is for scraping taz articles
- q=Russland
- dbShortcut=TAZ


## Warning
- Code makes to many server requests, you account will get blocked

In [1]:
# Import libraries

import csv
import re
import time
import os
import sys

from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from urllib.parse import urljoin

from threading import Thread

import pandas as pd

In [2]:
# Get chromedriver
driver = webdriver.Chrome()
driver.get('https://www.wiso-net.de/')

base_path = os.path.join(os.path.abspath(''), 'data')

# Log in now!!

**Before executing next cell, log into wiso-net with university account**

In [3]:
# This requires that you logged into the WisoNet with your credentials in the new chrome window
# Be aware that wiso saves the search inputs, so multiple executions may result in unwanted behavior
driver.get('https://www.wiso-net.de/dosearch?dbShortcut=TAZ')
driver.find_element(by=By.ID, value='field_q').send_keys('Russland')
driver.find_element(by=By.ID, value='field_q').send_keys(Keys.ENTER)

In [5]:
number_of_iterations = 4

base_url = "https://www.wiso-net.de/"

df = pd.DataFrame(columns=['title','author', 'category', 'date', 'text'])
df.index.name = 'row_id'

for i in tqdm(position=0,leave=True,iterable=range(number_of_iterations)):
    
    # for safety always set the driver to the first and main window
    driver.switch_to.window(driver.window_handles[0])

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    try:
        urls = soup.select('tr.hitlist_item td.boxCol4 a')
    except:
        print(f"No more urls")
        break

    for j in range(len(urls)):
        url = urls[j]['href']
        full_url = urljoin(base_url, url)

        driver.execute_script(f"window.open('{full_url}', '_blank');")
        time.sleep(0.1)

    assert len(driver.window_handles) == len(urls) + 1

    for j in range(len(urls)):
        
        driver.switch_to.window(driver.window_handles[-1])

        try:
            wait = WebDriverWait(driver, 10)
            wait.until(EC.presence_of_element_located(
                (By.CSS_SELECTOR, 'pre.gray')))
        except:
            print(f"Couldn't load article {j+1} from page {i+1}")
            print("trying again")
            try:
                driver.refresh()
                wait = WebDriverWait(driver, 10)
                wait.until(EC.presence_of_element_located(
                    (By.CSS_SELECTOR, 'pre.gray')))
            except:
                print(f"still not able, skipping article {j+1} from page {i+1}")
                driver.close()
                continue
            print("refresh worked")
        
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        try:
            title = soup.select_one('div#content pre.boldLarge').text

            author = soup.select_one('div#content pre.italic')
            if author is not None:
                author = author.text

            text = soup.select('div#content pre.text:not(.permaLink)')
            text = [t.text.strip().replace("\n"," ") for t in text]
            text = ','.join(text)

            category = soup.select_one('div#content pre.gray').text
            date = category.split('/')[0].rstrip()
            category = category.split('/')[1]
            category = category.rstrip()
            date = re.search('\d{2}.\d{2}.\d{4}', date).group(0)
        except:
            print(f"Couldn't load page {j+1} from page {i+1}")
            driver.close()
            continue

        df2 = pd.DataFrame({'title': title, 'author': author,'category':str(category), 'date': date, 'text': text},index=[0])
        df2 = df2.astype(str)
        
        df = pd.concat([df, df2], ignore_index=True)
        driver.close()

    if i % 2 == 0 and i != 0:
        df.to_csv(os.path.join(base_path, f'taz_articles_{i}.csv'),quoting=csv.QUOTE_NONNUMERIC)
        print(f"Saved {i*10} articles")


    driver.switch_to.window(driver.window_handles[0])
    try:
        driver.find_element(By.CSS_SELECTOR, 'div.innerModulePager a.nextLink').click()
    except:
        print(f"Couldn't load page {i}")
        continue
    wait = WebDriverWait(driver, 10)
    element = wait.until(
        lambda d: d.find_element(By.CSS_SELECTOR,'span.currentStep') and d.find_element(By.CSS_SELECTOR,'span.currentStep').text == str(i+2)
        )
df.to_csv(os.path.join(base_path, f'taz_articles_final.csv'),quoting=csv.QUOTE_NONNUMERIC)
print("finished scraping taz")


  0%|          | 0/4 [00:00<?, ?it/s]

Couldn't load article 6 from page 1
trying again
still not able, skipping article 6 from page 1
Couldn't load article 7 from page 1
trying again
still not able, skipping article 7 from page 1
Couldn't load article 8 from page 1
trying again
refresh worked


 25%|██▌       | 1/4 [01:30<04:30, 90.09s/it]

Couldn't load article 1 from page 2
trying again
still not able, skipping article 1 from page 2


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=115.0.5790.111)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x002FA813+48355]
	(No symbol) [0x0028C4B1]
	(No symbol) [0x00195358]
	(No symbol) [0x0017D293]
	(No symbol) [0x001DE37B]
	(No symbol) [0x001E2842]
	(No symbol) [0x001DA703]
	(No symbol) [0x001B82DC]
	(No symbol) [0x001B93DD]
	GetHandleVerifier [0x0055AABD+2539405]
	GetHandleVerifier [0x0059A78F+2800735]
	GetHandleVerifier [0x0059456C+2775612]
	GetHandleVerifier [0x003851E0+616112]
	(No symbol) [0x00295F8C]
	(No symbol) [0x00292328]
	(No symbol) [0x0029240B]
	(No symbol) [0x00284FF7]
	BaseThreadInitThunk [0x75AE7D59+25]
	RtlInitializeExceptionChain [0x7719B79B+107]
	RtlClearBits [0x7719B71F+191]
	(No symbol) [0x00000000]


In [12]:
df

,title,author,category,date,text
0,Wege für Weizen,Lisa Schneider,politik,05.08.2023,"die erklärung,Russland hat den Getreidedeal mi..."
1,"Traumhaftes, geliebtes, zerstörtes Odessa",Erica Zingher,Gesellschaft,05.08.2023,"Erica Zingher Grauzone,Dieser Tage denke ich v..."
2,"Zeitenwende, Zerrbilder, Zerreißprobe",Dominic Johnson,politik,05.08.2023,Ist Nigers Putsch eine „Vollendung der Souverä...
3,Das Europa der Rechten,Claus Leggewie,politik,05.08.2023,Im kommenden Jahr werden die Abgeordneten des ...
4,"Woher ich weiß, dass Sahne auf Bratkartoffeln ...",Ulrike Winkelmann,politik,05.08.2023,"Ulrike Winkelmann Ernsthaft?,Mein Freund Fred ..."
...,...,...,...,...,...
394,Fragile Gerechtigkeit,Gemma Teres Arilla,Schwerpunkt,05.06.2023,Seit dem großangelegten Gefangenenaustausch im...
395,Mächtige Gegner für die Umwelt,Rudolf Balmer,Meinung und Diskussion,05.06.2023,Rudolf Balmer zur UN-Konferenz zum Plastikmüll...
396,"Im Namen des Gesetzes schuldig, dennoch frei",Juri Larin,Schwerpunkt,05.06.2023,Warum die Ukraine einen verurteilten russische...
397,Saat der Hoffnung: Mutige Landwirte in der Ukr...,Tatjana Milimko,Schwerpunkt,05.06.2023,"„krieg und frieden“: ein tagebuch,Aus Odessa T..."


In [13]:
df.to_csv(f"G:\coding\PcToLaptop\CER\project\data\\articlesx.csv",quoting=csv.QUOTE_NONNUMERIC)

In [30]:
path = os.path.join(os.path.abspath(''), 'data')
df.to_csv(os.path.join(path, 'taz_articles2.csv'))